## Packages - Installation & Import

In [ ]:
#surprise package installation
!pip install surprise

In [1]:
#import relevant packages
import random
import pandas as pd
import time
import sys
import pickle
from surprise import Dataset, NormalPredictor, Reader,SVDpp,KNNBasic,accuracy,CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split



colnames=['time', 'userid', 'movieid', 'ratings'] 
df = pd.read_csv('kafka_log_sample.csv', names=colnames, header=None)

### Taking the top 1000 popular movies

In [2]:
top_1000_movies=df.groupby('movieid').size().sort_values(ascending=False)[:1000].index.tolist()
# with open('top_1000_movie_list_final.pkl', 'wb') as file:
#     pickle.dump(top_1000_movies, file)

## Load & Split Data 

In [3]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
dataset = Dataset.load_from_df(df[["userid", "movieid", "ratings"]], reader)

trainset, testset = train_test_split(dataset, test_size=0.25)



## Algorithm : Singular Value Decompostition

### Evaluation on entire test data

In [4]:
algo =  SVDpp()

# Train the algorithm on the trainset, and predict ratings for the testset
%time algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

CPU times: user 6.01 s, sys: 63 ms, total: 6.08 s
Wall time: 6.12 s
RMSE: 0.6831


0.6830529639938385

## Avoiding Pitfalls in Evaluation

### 1) Evaluation on test data across Splices - Gender

### Gender

In [5]:
#import user gender details
test_user_details = pd.read_csv('user_details.csv')
# divide testset into male and female
male = list(test_user_details[test_user_details['gender'] == 'M']['user_id'])
female = list(test_user_details[test_user_details['gender'] == 'F']['user_id'])

testsetsample = random.sample(testset,2000)
test_male = [i for i in testsetsample if i[0] in male]
test_female = [i for i in testsetsample if i[0] in female]


#### Results expected to be consistent across genders

#### Male

In [8]:
predictions_m = algo.test(test_male)
accuracy.rmse(predictions_m)

RMSE: 0.6812


0.6812067194355739

#### Female

In [9]:
predictions_f = algo.test(test_female)
accuracy.rmse(predictions_f)

RMSE: 0.6829


0.682862691182107

### 2) K-Fold Crossvalidation 

#### Results expected to be consistent across folds

In [11]:
from surprise.model_selection import cross_validate

cross_validate(algo,dataset,measures=['RMSE'],cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6830  0.6814  0.6811  0.6823  0.6831  0.6822  0.0008  
Fit time          6.39    6.59    6.58    5.97    5.27    6.16    0.50    
Test time         1.93    1.69    1.53    1.37    1.20    1.54    0.25    


{'test_rmse': array([0.68295863, 0.6814358 , 0.68113572, 0.68234366, 0.68307372]),
 'fit_time': (6.3873066902160645,
  6.5910162925720215,
  6.58030891418457,
  5.974684000015259,
  5.270564079284668),
 'test_time': (1.9346349239349365,
  1.686007022857666,
  1.5266940593719482,
  1.3721983432769775,
  1.2038259506225586)}